In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
os.getcwd()

'd:\\My Projects\\Book Recommender\\notebooks'

In [3]:
%cd ..

d:\My Projects\Book Recommender


In [4]:
books = pd.read_csv(r"artifacts/semantic_booksdata/clean_books_dataset.csv", sep=";", encoding="iso8859", on_bad_lines="skip")
books.head(3)

,isbn13,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tagged_description
0,9780002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883: A NOVEL THAT READERS and critic...
1,9780002261982,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982: A new 'Christie for Christmas' ...
2,9780006178736,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736: A memorable, mesmerizing heroin..."


In [5]:
books.shape

(5158, 12)

In [6]:
books["categories"].value_counts().reset_index().sort_values("count", ascending=False)

,categories,count
0,Fiction,2071
1,Juvenile Fiction,406
2,Biography & Autobiography,319
3,History,210
4,Literary Criticism,125
...,...,...
242,Autism,1
241,English,1
240,"Everest, Mount (China and Nepal)",1
239,"Detective and mystery stories, American",1


In [7]:
books["categories"].value_counts().reset_index().sort_values("count", ascending=False).query("count >= 25")

,categories,count
0,Fiction,2071
1,Juvenile Fiction,406
2,Biography & Autobiography,319
3,History,210
4,Literary Criticism,125
5,Religion,119
6,Philosophy,118
7,Comics & Graphic Novels,113
8,Drama,83
9,Juvenile Nonfiction,64


In [8]:
category_mapping = {"Fiction" : "Fiction",
                    "Juvenile Fiction" : "Children's Fiction",
                    "Biography & Autobiography" : "Narative Nonfiction",
                    "History" : "History",
                    "Literary Critism" : "Nonfiction",
                    "Literary Collections" : "Arts",
                    "Philosophy" : "Nonfiction",
                    "Religion" : "Nonfiction",
                    "Comics & Graphic Novels" : "Fiction",
                    "Drama": "Arts",
                    "Juvenile Nonfiction" : "Children's NonFiction",
                    "Science" : "Nonfiction",
                    "Poetry" : "Arts",
                    "Business & Economics": "Nonfiction",
                    "Health & Fitness": "Nonfiction",
                    "Travel" : "Nonfiction",
                    "Family & Relationships" : "Nonfiction",
                    "Body, Mind & Spirit" : "Nonfiction",
                    "Cooking" : "Arts",
                    "Performing Arts" : "Arts",
                    "Art" : "Arts",
                    "Language Arts & Disciplines" : "Arts"
                }

In [9]:
books["new_categories"] = books["categories"].map(category_mapping)

In [10]:
books["new_categories"].value_counts().reset_index()

,new_categories,count
0,Fiction,2184
1,Nonfiction,463
2,Children's Fiction,406
3,Narative Nonfiction,319
4,Arts,311
5,History,210
6,Children's NonFiction,64


In [11]:
books.isnull().sum()

isbn13                   0
title                    0
authors                 32
categories              29
thumbnail              156
description              0
published_year           0
average_rating           0
num_pages                0
ratings_count            0
title_subtitle           0
tagged_description       0
new_categories        1201
dtype: int64

In [ ]:
new_books = books.dropna(subset=["new_categories"], axis=0)

In [14]:
new_books.drop(["categories"], axis =1, inplace=True)

C:\Users\subin\AppData\Local\Temp\ipykernel_13764\3848988331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_books.drop(["categories"], axis =1, inplace=True)


In [15]:
new_books.head(2)

,isbn13,title,authors,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tagged_description,new_categories
0,9780002005883,Gilead,Marilynne Robinson,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883: A NOVEL THAT READERS and critic...,Fiction
2,9780006178736,Rage of angels,Sidney Sheldon,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736: A memorable, mesmerizing heroin...",Fiction


In [ ]:
book_categories = new_books["new_categories"].unique()
book_categories

In [ ]:
import os
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0, api_key=api_key)

True

In [37]:
description = new_books["description"].iloc[3]
description

"Joe and his parents are enjoying a summer holiday by the sea at the Ocean Star Hotel. The sky is bright blue, the sun shines and Joe loves all that the seaside has to offer. But when the fog rolls in and rain falls Joe begins to wish that he was back at home again. Things change, however, when the owner of the hotel invites Joe to share in a magical world, only a few steps away. The loft is black as night but then above Joe's head a thousand tiny stars begin to sparkle and in the distance he hears the chug-chug-chug of a model train. A whole world is soon to open up before Joe's eyes, a world of snow-capped mountains, great deserts, and rocking fishing boats."

In [40]:
category = new_books["new_categories"].iloc[3]
category

"Children's Fiction"

In [38]:
template = PromptTemplate(
    input_variables = ["description"],
    template = """You are an expert book classification assistant. Given the description of book your task is to categorize the book into one of the categories:- ['Fiction', 'Children's Fiction', 'Arts', 'Narative Nonfiction', 'History', 'Nonfiction', 'Children's NonFiction']

    Book Description: {description}

    Expected Output: Category of the book.
"""
)

In [ ]:
prompt = template.invoke({"description": description})

response = llm.invoke(prompt)
print(response.content)

Children's Fiction
